In [16]:
import pandas as pd
import time
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        _, (h, _) = self.lstm(x)
        out = self.fc(h[-1])
        return out

# Read the dataset
data = pd.read_csv("../DataSet/cs448b_ipasn.csv")

# Extract the required columns
dates = pd.to_datetime(data["date"])
local_ips = data["l_ipn"]
remote_asns = data["r_asn"]
flow_counts = data["f"]

# Token bucket parameters
capacity = 1000
tokens = capacity
rate = 100  # tokens per second

# LSTM parameters
input_size = 1
hidden_size = 32
output_size = 1
num_epochs = 100
learning_rate = 0.001

# Prepare the flow rate data for LSTM training
flow_rates = flow_counts.values.reshape(-1, 1)
scaler = MinMaxScaler()
flow_rates = scaler.fit_transform(flow_rates)
flow_rates = torch.FloatTensor(flow_rates)

# Train the LSTM model
model = LSTMModel(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(flow_rates[:-1].unsqueeze(2))
    loss = criterion(outputs, flow_rates[1:])
    loss.backward()
    optimizer.step()

# Evaluation of the token bucket model
total_tokens = 0
total_flow_count = 0
total_matched_flow_count = 0

for i in range(len(data)):
    current_time = time.time()
    flow_count = flow_counts[i]
    token_requirement = flow_count

    # Wait until tokens are available
    while tokens < token_requirement:
        time.sleep(0.1)
        elapsed_time = time.time() - current_time
        tokens += elapsed_time * rate

        # Adjust tokens based on the flow rate (adaptive component using LSTM)
        flow_rate_input = torch.FloatTensor([[flow_count]])
        predicted_flow_rate = model(flow_rate_input.unsqueeze(2)).item()
        token_requirement = int(predicted_flow_rate * elapsed_time)

        # Ensure tokens don't exceed the capacity
        if tokens > capacity:
            tokens = capacity

    # Tokens are available, process the flow
    tokens -= token_requirement

    # Evaluation
    total_tokens += token_requirement
    total_flow_count += flow_count
    if token_requirement == flow_count:
        total_matched_flow_count += 1

    # Print information about the flow
    # date = dates[i].strftime("%Y-%m-%d")
    # local_ip = local_ips[i]
    # remote_asn = remote_asns[i]
    # print(f"Flow on {date}: Local IP={local_ip}, Remote ASN={remote_asn}, Flow Count={flow_count}")

/Users/shadhrushswaroop/Library/Python/3.9/lib/python/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([20802, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


KeyboardInterrupt: 

In [ ]:
# Calculate evaluation metrics
token_utilization = (total_tokens / (rate * len(data))) * 100
accuracy = (total_matched_flow_count / total_flow_count) * 100

print(f"\n--- Evaluation Results ---")
print(f"Token Utilization:{token_utilization}")
print(f"Accuracy:{accuracy}")


--- Evaluation Results ---
Token Utilization:2.489929337114839
Accuracy:0.8139947934634729
